# [5장] 연합뉴스 타이틀 분류
## 1. 분류를 위한 기본 설정(p.115)
###★ 필요한 라이브러리
#### - 판다스(데이터 분석도구), 넘파이(수치계산), 시본과 맷플롯립(데이터 시각화)

In [ ]:
#판다스, 넘파이, 시본과 맷플롯립 불러오기(공식 문서의 별칭을 사용)
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### ★ 한글 폰트 설정을 위한 설치
#### - koreanize-matplotlib
#### - konlpy : 한국어 형태소 분석
#### - tqdm : 작업의 진행 상태 확인

In [ ]:
!pip install koreanize-matplotlib

In [ ]:
!pip install konlpy --upgrade

In [ ]:
!pip install tqdm --upgrade

In [ ]:
import koreanize_matplotlib
#그래프에 retina display 적용
%config InlineBackend.figure_format='retina'
pd.Series([1,3,5,-7,9]).plot(title="한글")

##2. 데이터 불러오기 : 데이콘에서 불러오기(p.117)
### 1) 해당 경로에 파일이 있는지 확인
### 2) 파일이 업다면 폴더를 생성한다
### 3) 생성한 폴더에 데이터를 다운로드한 뒤
### 4) open.zip 파일의 압축을 푼다


In [ ]:
import os
import platform

base_path="data/klue/"
file_name="dacon-klue-open-zip"

#파일이 있는지 확인하는 함수 작성
def file_exist_check(base_path) :
  #파일 경로 만들기
  if os.path.exists(f"{base_path}train_data.csv") : #해당 경로에 파일이 있다면
    print(f"{os.getcwd()}/{base_path} 경로에 파일이 있음")
    return
  if not os.path.exists(f"{base_path}train_data.csv") : #해당 경로에 파일이 없다면
    os.makedirs(base_path)
  if platform.system()=="Linux" :
    print(f"파일을 다운로드 하고 {base_path} 경로에 압축을 해제함")
    !wget https://bit.ly/{file_name}
    !unzip {file_name} -d {base_path}
    return
  else :
    print(f"""https://dacon.io/competitions/official/235747/data 에서 다운로드 해 실습 경로 {os.getcwd()}/{base_path}에 옮겨주세요.""")
    return

file_exist_check(base_path)

## 3. 데이터 전처리하기
### ★ 지도학습 준비
#### - 학습할 데이터 : train
#### - 예측할 데이터 : test


In [ ]:
#판다스를 활용해 데이터 프레임 형태로 데이터 불러오기
train=pd.read_csv(os.path.join(base_path, "train_data.csv")) #경로 합치기 "data/clue/train_data.csv"
test=pd.read_csv(os.path.join(base_path,"test_data.csv"))
#데이터의 크기 확인(행,열)
#.
#훈련 데이터의 컬럼 확인
#.
#테스트 데이터의 컬럼 확인
#.

In [ ]:
#토픽 불러오기
topic=pd.read_csv(os.path.join(base_path, "topic_dict.csv"))
topic

In [ ]:
#토픽 데이터의 값 확인
topic["topic"].values

In [ ]:
train.head()

In [ ]:
test.head()

### ★ 데이터 전처리 과정
#### - 전체적인 데이터 특성 파악 : 문자 길이, 단어의 등장 빈도 등 (시각화)
#### - 불필요한 기호를 제거 후 중요한 정보를 담은 내용 추출
#### * 단, 트레이닝 데이터와 테스트 데이터를 정렬 하여 전처리한 후 다시 나눈다.

In [ ]:
#전처리를 위한 데이터 병합 1: train과 test데이터는 단순히 아래에 이어서 연결된다(행방향)
raw=pd.concat([train,test])
print(raw.shape)
raw.head()
raw.tail()

In [ ]:
#전처리를 위한 데이터 병합 2: raw(train+test)에 topic 병합하되, 각 데이터의 고유값(topic_idx)를 기준으로 병합
df=raw.merge(topic, how="left")
df.shape

In [ ]:
df.head()

In [ ]:
df.tail()

### ★정답값 빈도수 확인
#### 트레이닝 데이터에서 제공하는 정답값의 빈도수를 구해서 예측하는 값의 빈도수가 비슷한지 차이가 있는지 확인하기

In [ ]:
#topic_idx에 따른 빈도수 구하기
df["topic_idx"].value_counts()

In [ ]:
!apt-get update -qq   # 폰트 설정을 위한 코드
!apt-get install fonts-nanum* -qq

In [ ]:
#topic_idx에 따른 빈도수 시각화하기
sns.countplot(data=df, y="topic")


### ★문자 길이 확인
#### - 음절의 길이(len)
#### - 단어의 빈도(word_count)
#### - 중복없는 어절의 길이(unique_word_count)

In [ ]:
df["len"]=df["title"].apply(lambda x : len(x))
df["word_count"]=df["title"].apply(lambda x : len(x.split()))
df["unique_word_count"]=df["title"].apply(lambda x : len(set(x.split())))

In [ ]:
df.head()

### ★ 맷플롯립과 시본을 이용해 히스토그램으로 시각화


In [ ]:
fig, axes=plt.subplots(1,3,figsize=(15,2))
sns.histplot(df["len"], ax=axes[0])
sns.histplot(df["word_count"],ax=axes[1])
sns.histplot(df["unique_word_count"],ax=axes[2])

In [ ]:
df[["len","word_count","unique_word_count"]].describe()

### ★주제별 글자와 단어의 빈도 확인
#### - 시본의 displot으로 확인

In [ ]:
sns.displot(data=df, x="len", hue="topic", col="topic", col_wrap=2, aspect=5, height=2)

## 4. 문자 전처리 하기(p.130)
### ★ 단어 가방 모형 :  단어 사전이 너무 많아지면 학습 속도가 오래 걸리거나 과적합 발생
#### - 불필요한 문자 제거
#### - 형태소 분석기를 사용해 불필요한 조사 제거
#### - 형태소를 표기해서 같은 단어지만 다른 의미를 갖는 단어 구분하기


### ★ 숫자 제거
#### - 이번 예제에서는 숫자에 큰 의미가 없으므로 제거함.


In [ ]:
import re
df["title"]=df["title"].str.replace("[0-9]","",regex=True)
df.head()

### ★영문자는 모두 소문자로 변경
#### - 파이썬은 대. 소문자를 다른 단어로 인식하여 다르게 분류함.
#### - 이 예제에서는 대소문자를 같은 단어로 인식하여야 하므로 모두 소문자로 변환

In [ ]:
df["title"]=df["title"].str.lower()
df.head()

### ★ 형태소 분석기로 조사, 어미, 구두점 제거
#### - KoNLPy을 이용하여 품사 태깅 / 조사, 어미, 구두점 제거 하기
#### * 품사 태깅 작업은 오래 걸린다.(tqdm으로 진행상태 확인)
#### * Kkma, Komoran, Hannanum, Okt, Mecab 형태소 분석기 중 Okt 사용

##### 형태소 분석 간단 샘플

In [ ]:
from konlpy.tag import Okt

small_text="아버지가 방에 들어가신다."
%time Okt().pos(small_text)

In [ ]:
from konlpy.tag import Okt
okt=Okt()

#조사, 어미, 구두점 제거, 어간 추출
def okt_clean(text) :
  clean_text=[]
  for word in okt.pos(text, stem=True) :
    if word[1] not in['Josa','Emoi','Punctuation'] :
      clean_text.append(word[0])
    return " ".join(clean_text)

from tqdm import tqdm
tqdm.pandas()

train['title']=train['title'].progress_map(okt_clean)
test['title']=test['title'].progress_map(okt_clean)


### ★불용어 제거
#### - split으로 문자열을 분리해 토큰 형태로 생성한다
#### - 불용어 목록을 리스트로 만든다
#### - 문서의 토큰이 불용어에 해당되지 않는 것을 리스트로 반환한다.
#### - 불용어를 제거했다면 다시 문장 하나로 합쳐 준다.

In [ ]:
def remove_stopwords(text) :
  tokens=text.split(' ') #공백 단위로 자르기
  stops=['합니다','하는','할','하고','한다','그리고','입니다','그','등','이런',' 것',' 및',' 제 ',' 더 ']
  meaningful_words=[w for w in tokens if not w in stops]
  return ' '.join(meaningful_words)

  df['title']=df['title'].map(remove_stopwords)


## 7. 트레이닝 데이터, 시험 데이터 분리하기
#### - topic_idx에 따라 분류할 수 있다.
#### - 판다스pandas의 notnull(), isnull() 활용

In [ ]:
label_name="topic_idx"

train=df[df[label_name].notnull()]
test=df[df[label_name].isnull()]

train.head()

In [ ]:
test.head()

In [ ]:
 train.shape, test.shape

In [ ]:
X_train=train["title"]
X_test=test["title"]

print(type(X_train), type(X_test))

In [ ]:
X_train.shape, X_test.shape

In [ ]:
#트레이닝 데이터 세트의 topic별 갯수 확인
y_train=train[label_name]
y_train.value_counts()

In [ ]:
#테스트 데이터 세트의 topic별 갯수 확인
y_test=test[label_name]
y_test.value_counts()

## 6. 단어 벡터화하기
#### - 사이킷런(sklearn)의 TFidVectorizer를 이용하여 단어가방벡터 만들기

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect=TfidfVectorizer(tokenizer=None,
                         ngram_range=(1,2),
                         min_df=3,
                         max_df=0.95)

#트레이닝용 데이터의 title컬럼의 값을 벡터화
tfidf_vect.fit(X_train)
type(X_train)

In [ ]:
train_feature_tfidf=tfidf_vect.transform(X_train)      #벡터 행렬로 변환
test_feature_tfidf=tfidf_vect.transform(X_test)

train_feature_tfidf.shape, test_feature_tfidf.shape

In [ ]:
vocab=tfidf_vect.get_feature_names_out()
print(len(vocab))
vocab[:10]

In [ ]:
dist=np.sum(train_feature_tfidf, axis=0)

vocab_count=pd.DataFrame(dist, columns=vocab)
vocab_count

In [ ]:
vocab_count.T[0].sort_values(ascending=False).head(50).plot.bar(figsize=(15,4))

## 7. 학습과 예측하기
### 1) 랜덤 포레스트 분류기 생성하기(p.144)
#### - 랜덤 포레스트 : 여러 개의 결정 트리를 만들어 학습과 예측을 함
#### - 사이킷런 라이브러리 이용

In [ ]:
#RandomForestClassifier를 불러 온다
#.
#랜덤 포레스트 분류기를 사용하여 결정트리 100,모든 CPU의 코어를 사용, random 시드값 42로 설정하기
#.
model

In [ ]:
from sklearn.model_selection import cross_val_predict
y_pred=cross_val_predict(model, train_feature_tfidf, y_train, cv=3, n_jobs=-1, verbose=1)

In [ ]:
#교차검증 정확도
valid_accuracy=(y_pred==y_train).mean()
valid_accuracy

In [ ]:
df_accuracy=pd.DataFrame({"pred":y_pred, "train":y_train})
df_accuracy["accuracy"]=(y_pred==y_train)

### 2)생성된 랜덤 포레스트 분류기로 학습하고, 평가하기
#### - fit()으로 학습하고, predict()로 예측함

In [ ]:
#학습 : fit
#.

In [ ]:
#예측 : predict
#.
#10개의 타이틀을 예측한 결과 확인
y_predict[:10]